In [ ]:
import os
import random
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from dataclasses import dataclass
block_plot = False

SEED_VALUE = 42
random.seed(SEED_VALUE)
np.random.seed(SEED_VALUE)
tf.random.set_seed(SEED_VALUE)


print(tf.__version__)


In [ ]:
def sys_config():
    gpu_devices = tf.config.list_physical_devices('GPU')
    print(gpu_devices)

In [ ]:
sys_config()